In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '957116', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41846,CN0100100000000,1,1,"Autauga County, AL",Dec-20(p),"25,029","24,237",792,3.2,01001
41847,CN0100300000000,1,3,"Baldwin County, AL",Dec-20(p),"91,638","88,526","3,112",3.4,01003
41848,CN0100500000000,1,5,"Barbour County, AL",Dec-20(p),"9,629","9,109",520,5.4,01005
41849,CN0100700000000,1,7,"Bibb County, AL",Dec-20(p),"8,539","8,193",346,4.1,01007
41850,CN0100900000000,1,9,"Blount County, AL",Dec-20(p),"24,585","24,005",580,2.4,01009


In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
45060,CN7214500000000,72,145,"Vega Baja Municipio, PR",Dec-20(p),"12,389","10,695",1694,13.7,72145
45061,CN7214700000000,72,147,"Vieques Municipio, PR",Dec-20(p),"2,510","2,195",315,12.5,72147
45062,CN7214900000000,72,149,"Villalba Municipio, PR",Dec-20(p),"6,754","5,799",955,14.1,72149
45063,CN7215100000000,72,151,"Yabucoa Municipio, PR",Dec-20(p),"7,920","6,917",1003,12.7,72151
45064,CN7215300000000,72,153,"Yauco Municipio, PR",Dec-20(p),"9,284","7,939",1345,14.5,72153


In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(100)

,date,county,state,fips,cases,deaths
1121570,2021-03-14,Webster,West Virginia,54101.0,379,6.0
1121571,2021-03-14,Wetzel,West Virginia,54103.0,1107,23.0
1121572,2021-03-14,Wirt,West Virginia,54105.0,362,2.0
1121573,2021-03-14,Wood,West Virginia,54107.0,7254,155.0
1121574,2021-03-14,Wyoming,West Virginia,54109.0,1772,38.0
1121575,2021-03-14,Adams,Wisconsin,55001.0,1769,15.0
1121576,2021-03-14,Ashland,Wisconsin,55003.0,1255,16.0
1121577,2021-03-14,Barron,Wisconsin,55005.0,5626,76.0
1121578,2021-03-14,Bayfield,Wisconsin,55007.0,1147,19.0
1121579,2021-03-14,Brown,Wisconsin,55009.0,33178,260.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
373,03/08/2021,3154,1275,251,62,9,2671,3718,243,56,61,570,230,70,14,1,483,675,67,12,12,1039,368,59,14,1,817,1119,67,15,15,461,199,46,10,1,396,573,41,10,10,836,402,61,17,0,754,1063,56,15,16,248,70,15,7,0,221,284,12,4,5,11000
374,03/09/2021,3014,1172,237,63,9,2671,3733,240,57,64,523,193,65,16,1,480,671,68,12,13,983,364,59,16,0,828,1140,65,15,15,489,203,43,11,0,398,576,41,11,11,797,347,66,19,0,748,1060,57,16,16,220,57,4,1,0,216,281,10,4,4,11000
375,03/10/2021,2615,1210,204,47,13,2600,3667,231,55,62,451,209,57,15,1,457,645,65,12,13,797,340,60,12,0,811,1119,62,15,15,440,178,27,4,1,389,564,38,9,9,717,385,52,12,1,734,1056,56,15,15,210,88,8,4,0,209,277,9,4,4,11000
376,03/11/2021,2103,1106,83,43,12,2473,3541,206,53,61,313,203,23,12,0,422,613,58,12,13,688,318,26,15,0,780,1085,55,15,15,308,154,9,3,0,365,536,33,8,9,641,344,18,10,0,709,1037,52,14,14,150,75,7,3,0,195,261,9,4,4,11000
377,03/12/2021,1470,1034,9,32,27,2289,3364,176,50,62,195,164,8,9,0,376,563,49,13,13,475,288,1,16,0,726,1034,47,15,15,206,174,0,4,0,340,511,28,7,8,467,318,0,3,0,665,1000,45,13,13,121,63,0,0,0,180,245,8,3,3,11000


In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
373,03/08/2021,570,14,New York,Bronx,36005.0
374,03/09/2021,523,16,New York,Bronx,36005.0
375,03/10/2021,451,15,New York,Bronx,36005.0
376,03/11/2021,313,12,New York,Bronx,36005.0
377,03/12/2021,195,9,New York,Bronx,36005.0


In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
373,03/08/2021,1039,14,Kings,New York,36047.0
374,03/09/2021,983,16,Kings,New York,36047.0
375,03/10/2021,797,12,Kings,New York,36047.0
376,03/11/2021,688,15,Kings,New York,36047.0
377,03/12/2021,475,16,Kings,New York,36047.0


In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,9813,184


In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
364,02/27/2021,612,19,New York,Queens,36081.0,2021-02-27
365,02/28/2021,562,23,New York,Queens,36081.0,2021-02-28
366,03/01/2021,1001,22,New York,Queens,36081.0,2021-03-01
367,03/02/2021,834,17,New York,Queens,36081.0,2021-03-02
368,03/03/2021,816,15,New York,Queens,36081.0,2021-03-03
369,03/04/2021,816,20,New York,Queens,36081.0,2021-03-04
370,03/05/2021,774,10,New York,Queens,36081.0,2021-03-05
371,03/06/2021,643,14,New York,Queens,36081.0,2021-03-06
372,03/07/2021,557,14,New York,Queens,36081.0,2021-03-07
373,03/08/2021,836,17,New York,Queens,36081.0,2021-03-08


In [45]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
328,01/22/2021,Richmond,New York,36085.0,327,3.0,2021-01-22
329,01/23/2021,Richmond,New York,36085.0,245,10.0,2021-01-23
330,01/24/2021,Richmond,New York,36085.0,179,6.0,2021-01-24
331,01/25/2021,Richmond,New York,36085.0,357,7.0,2021-01-25
332,01/26/2021,Richmond,New York,36085.0,279,2.0,2021-01-26
333,01/27/2021,Richmond,New York,36085.0,279,5.0,2021-01-27
334,01/28/2021,Richmond,New York,36085.0,268,7.0,2021-01-28
335,01/29/2021,Richmond,New York,36085.0,233,6.0,2021-01-29
336,01/30/2021,Richmond,New York,36085.0,162,4.0,2021-01-30
337,01/31/2021,Richmond,New York,36085.0,218,6.0,2021-01-31


In [46]:
print (data_date_dt)

2021-03-15 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1118423,1001.0,95.0
1118424,1003.0,294.0


In [48]:
Kings_death.head(2)

,fips,deaths
0,36047.0,7612


In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1121425,51690.0,70.0
1121426,51115.0,12.0
1121427,51117.0,59.0
1121428,51119.0,18.0
1121429,51121.0,90.0
1121430,51125.0,11.0
1121431,51127.0,13.0
1121432,51700.0,202.0
1121433,51710.0,227.0
1121434,51131.0,35.0


In [50]:
covid_death.shape

(3252, 2)

In [51]:
print (County_covid_death)

{1001.0: 95.0, 1003.0: 294.0, 1005.0: 53.0, 1007.0: 58.0, 1009.0: 129.0, 1011.0: 39.0, 1013.0: 66.0, 1015.0: 301.0, 1017.0: 112.0, 1019.0: 42.0, 1021.0: 106.0, 1023.0: 23.0, 1025.0: 56.0, 1027.0: 54.0, 1029.0: 41.0, 1031.0: 105.0, 1033.0: 125.0, 1035.0: 25.0, 1037.0: 24.0, 1039.0: 108.0, 1041.0: 55.0, 1043.0: 184.0, 1045.0: 108.0, 1047.0: 144.0, 1049.0: 178.0, 1051.0: 196.0, 1053.0: 73.0, 1055.0: 335.0, 1057.0: 58.0, 1059.0: 82.0, 1061.0: 71.0, 1063.0: 32.0, 1065.0: 72.0, 1067.0: 42.0, 1069.0: 267.0, 1071.0: 105.0, 1073.0: 1425.0, 1075.0: 33.0, 1077.0: 216.0, 1079.0: 92.0, 1081.0: 163.0, 1083.0: 142.0, 1085.0: 52.0, 1087.0: 45.0, 1089.0: 479.0, 1091.0: 58.0, 1093.0: 98.0, 1095.0: 217.0, 1097.0: 753.0, 1099.0: 39.0, 1101.0: 537.0, 1103.0: 258.0, 1105.0: 27.0, 1107.0: 57.0, 1109.0: 74.0, 1111.0: 41.0, 1113.0: 34.0, 1117.0: 226.0, 1115.0: 230.0, 1119.0: 31.0, 1121.0: 167.0, 1123.0: 143.0, 1125.0: 431.0, 1127.0: 268.0, 1129.0: 38.0, 1131.0: 26.0, 1133.0: 69.0, 2013.0: 2.0, 2016.0: 0.0, 202

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
1076536,2021-03-01,New Haven,Connecticut,9009.0,71998,1925.0,2021-03-01,-14 days,NaN
1079781,2021-03-02,New Haven,Connecticut,9009.0,72163,1926.0,2021-03-02,-13 days,165.0
1083027,2021-03-03,New Haven,Connecticut,9009.0,72368,1930.0,2021-03-03,-12 days,205.0
1086273,2021-03-04,New Haven,Connecticut,9009.0,72658,1933.0,2021-03-04,-11 days,290.0
1089518,2021-03-05,New Haven,Connecticut,9009.0,72947,1933.0,2021-03-05,-10 days,289.0
1092764,2021-03-06,New Haven,Connecticut,9009.0,72947,1933.0,2021-03-06,-9 days,0.0
1096010,2021-03-07,New Haven,Connecticut,9009.0,72947,1933.0,2021-03-07,-8 days,0.0
1099258,2021-03-08,New Haven,Connecticut,9009.0,73618,1937.0,2021-03-08,-7 days,671.0
1102504,2021-03-09,New Haven,Connecticut,9009.0,73834,1943.0,2021-03-09,-6 days,216.0
1105750,2021-03-10,New Haven,Connecticut,9009.0,74020,1946.0,2021-03-10,-5 days,186.0


In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,2680.0


In [57]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
328,01/22/2021,Richmond,New York,36085.0,327,3.0,2021-01-22
329,01/23/2021,Richmond,New York,36085.0,245,10.0,2021-01-23
330,01/24/2021,Richmond,New York,36085.0,179,6.0,2021-01-24
331,01/25/2021,Richmond,New York,36085.0,357,7.0,2021-01-25
332,01/26/2021,Richmond,New York,36085.0,279,2.0,2021-01-26
333,01/27/2021,Richmond,New York,36085.0,279,5.0,2021-01-27
334,01/28/2021,Richmond,New York,36085.0,268,7.0,2021-01-28
335,01/29/2021,Richmond,New York,36085.0,233,6.0,2021-01-29
336,01/30/2021,Richmond,New York,36085.0,162,4.0,2021-01-30
337,01/31/2021,Richmond,New York,36085.0,218,6.0,2021-01-31


In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,10.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-03-15 18:42:42.937359
3


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 95.0
1003.0 294.0
1005.0 53.0
1007.0 58.0
1009.0 129.0
1011.0 39.0
1013.0 66.0
1015.0 301.0
1017.0 112.0
1019.0 42.0
1021.0 106.0
1023.0 23.0
1025.0 56.0
1027.0 54.0
1029.0 41.0
1031.0 105.0
1033.0 125.0
1035.0 25.0
1037.0 24.0
1039.0 108.0
1041.0 55.0
1043.0 184.0
1045.0 108.0
1047.0 144.0
1049.0 178.0
1051.0 196.0
1053.0 73.0
1055.0 335.0
1057.0 58.0
1059.0 82.0
1061.0 71.0
1063.0 32.0
1065.0 72.0
1067.0 42.0
1069.0 267.0
1071.0 105.0
1073.0 1425.0
1075.0 33.0
1077.0 216.0
1079.0 92.0
1081.0 163.0
1083.0 142.0
1085.0 52.0
1087.0 45.0
1089.0 479.0
1091.0 58.0
1093.0 98.0
1095.0 217.0
1097.0 753.0
1099.0 39.0
1101.0 537.0
1103.0 258.0
1105.0 27.0
1107.0 57.0
1109.0 74.0
1111.0 41.0
1113.0 34.0
1117.0 226.0
1115.0 230.0
1119.0 31.0
1121.0 167.0
1123.0 143.0
1125.0 431.0
1127.0 268.0
1129.0 38.0
1131.0 26.0
1133.0 69.0
2013.0 2.0
2016.0 0.0
2020.0 159.0
2050.0 19.0
2997.0 0.0
2068.0 0.0
2070.0 1.0
2090.0 29.0
2100.0 0.0
2110.0 3.0
2122.0 19.0
2130.0 1.0
2150.0 5.0
2158.0 3.0
2170.

23009.0 34.0
23011.0 53.0
23013.0 5.0
23015.0 2.0
23017.0 54.0
23019.0 87.0
23021.0 2.0
23023.0 6.0
23025.0 27.0
23027.0 19.0
23029.0 14.0
23031.0 124.0
24001.0 201.0
24003.0 552.0
24005.0 1315.0
24510.0 909.0
24009.0 73.0
24011.0 21.0
24013.0 216.0
24015.0 124.0
24017.0 164.0
24019.0 45.0
24021.0 287.0
24023.0 61.0
24025.0 248.0
24027.0 220.0
24029.0 44.0
24031.0 1441.0
24033.0 1352.0
24035.0 40.0
24039.0 34.0
24037.0 117.0
24041.0 35.0
nan 52.0
24043.0 259.0
24045.0 146.0
24047.0 92.0
25001.0 422.0
25003.0 265.0
25005.0 1574.0
25007.0 0.0
25009.0 2223.0
25011.0 101.0
25013.0 1395.0
25015.0 274.0
25017.0 3536.0
25019.0 2.0
25021.0 1667.0
25023.0 1341.0
25025.0 1749.0
nan 9.0
25027.0 2087.0
26001.0 24.0
26003.0 5.0
26005.0 118.0
26007.0 48.0
26009.0 20.0
26011.0 25.0
26013.0 34.0
26015.0 49.0
26017.0 297.0
26019.0 32.0
26021.0 247.0
26023.0 111.0
26025.0 237.0
26027.0 67.0
26029.0 13.0
26031.0 38.0
26033.0 43.0
26035.0 69.0
26037.0 72.0
26039.0 5.0
26041.0 85.0
26043.0 70.0
26045.0 158

48145.0 29.0
48147.0 105.0
48149.0 56.0
48151.0 14.0
48153.0 29.0
48155.0 9.0
48157.0 604.0
48159.0 26.0
48161.0 47.0
48163.0 45.0
48165.0 42.0
48167.0 373.0
48169.0 21.0
48171.0 48.0
48173.0 2.0
48175.0 13.0
48177.0 54.0
48179.0 54.0
48181.0 325.0
48183.0 333.0
48185.0 64.0
48187.0 198.0
48189.0 153.0
48191.0 14.0
48193.0 26.0
48195.0 23.0
48197.0 12.0
48199.0 101.0
48201.0 5487.0
48203.0 100.0
48205.0 2.0
48207.0 26.0
48209.0 232.0
48211.0 2.0
48213.0 172.0
48215.0 2704.0
48217.0 102.0
48219.0 100.0
48221.0 121.0
48223.0 107.0
48225.0 47.0
48227.0 99.0
48229.0 11.0
48231.0 154.0
48233.0 70.0
48235.0 1.0
48237.0 15.0
48239.0 30.0
48241.0 82.0
48243.0 3.0
48245.0 370.0
48247.0 15.0
48249.0 91.0
48251.0 343.0
48253.0 52.0
48255.0 31.0
48257.0 231.0
48259.0 66.0
48261.0 2.0
48263.0 2.0
48265.0 83.0
48267.0 6.0
48269.0 0.0
48271.0 6.0
48273.0 84.0
48275.0 15.0
48283.0 24.0
48277.0 150.0
48279.0 80.0
48281.0 31.0
48285.0 72.0
48287.0 36.0
48289.0 40.0
48291.0 166.0
48293.0 68.0
48295.0 12.

In [69]:
K_c.head()

,fips,cases,deaths
0,36047.0,9813,184


In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 9813}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 36.0
1001 156.0
1009 190.0
1013 76.0
1015 764.0
1017 29.0
1031 48.0
1033 46.0
1039 58.0
1043 171.0
1045 61.0
1051 259.0
1055 312.0
1067 22.0
1069 107.0
1071 83.0
1077 91.0
1079 39.0
1083 123.0
1089 497.0
1095 252.0
1097 578.0
1103 210.0
1111 40.0
1113 86.0
1115 306.0
1117 551.0
1121 192.0
2261 24.0
4021 1445.0
5009 57.0
5011 9.0
5033 106.0
5037 18.0
5045 232.0
5047 30.0
5051 203.0
5053 33.0
5063 24.0
5083 31.0
5085 149.0
5087 17.0
5115 68.0
5117 12.0
5121 33.0
5125 152.0
5131 253.0
5145 103.0
6007 161.0
6017 139.0
6023 139.0
6027 62.0
6061 412.0
6079 345.0
6089 146.0
6093 71.0
8019 15.0
8039 47.0
8047 7.0
8051 23.0
8057 1.0
8065 13.0
8067 178.0
8071 19.0
8075 36.0
8077 197.0
8083 24.0
8085 36.0
8093 22.0
8099 2.0
8101 304.0
8103 18.0
8119 49.0
8121 28.0
9005 371.0
9007 311.0
9011 423.0
9015 212.0
10001 409.0
12019 283.0
12033 695.0
12089 130.0
12101 1294.0
12109 452.0
12113 336.0
13015 411.0
13039 78.0
13045 155.0
13047 149.0
13057 880.0
13073 250.0
13077 346.0
13085 54.0
13113 1

36107 130.0
36111 678.0
36113 130.0
36115 121.0
36117 147.0
36121 134.0
37001 391.0
37009 30.0
37021 348.0
37023 147.0
37025 446.0
37027 137.0
37031 111.0
37035 341.0
37037 102.0
37049 267.0
37053 40.0
37055 28.0
37057 239.0
37059 85.0
37069 232.0
37071 464.0
37077 198.0
37087 76.0
37097 358.0
37101 609.0
37109 223.0
37115 55.0
37121 27.0
37123 56.0
37125 189.0
37127 270.0
37139 57.0
37141 172.0
37145 104.0
37151 496.0
37157 94.0
37159 299.0
37161 104.0
37167 179.0
37169 48.0
37171 108.0
37197 71.0
38001 0.0
38003 16.0
38007 0.0
38009 10.0
38011 10.0
38021 4.0
38025 3.0
38027 1.0
38031 8.0
38033 3.0
38041 6.0
38043 1.0
38049 14.0
38055 14.0
38057 9.0
38059 23.0
38065 3.0
38067 7.0
38069 8.0
38073 8.0
38075 1.0
38081 7.0
38091 2.0
38095 2.0
38097 18.0
38099 24.0
39003 98.0
39005 102.0
39007 89.0
39011 24.0
39013 187.0
39015 85.0
39017 741.0
39019 28.0
39021 102.0
39023 286.0
39025 366.0
39027 77.0
39029 130.0
39033 94.0
39037 50.0
39039 69.0
39043 165.0
39045 290.0
39051 78.0
39053 36.0

29017 71.0
29019 211.0
29027 346.0
29035 100.0
29053 194.0
29065 86.0
29067 127.0
29077 302.0
29093 435.0
29095 6489.0
29101 982.0
29111 207.0
29117 385.0
29123 265.0
29135 85.0
29147 46.0
29163 275.0
29165 840.0
29169 442.0
29175 802.0
29179 2.0
29187 1129.0
29201 810.0
29207 161.0
29209 300.0
29211 105.0
29213 553.0
29225 480.0
30011 1.0
30019 2.0
30037 0.0
30039 0.0
30055 2.0
30057 16.0
30065 2.0
30077 4.0
30079 0.0
30087 19.0
30089 13.0
30093 88.0
30097 8.0
30109 1.0
31009 1.0
31013 12.0
31015 1.0
31017 2.0
31029 9.0
31045 23.0
31049 7.0
31055 1008.0
31057 1.0
31061 4.0
31063 0.0
31069 3.0
31071 3.0
31073 2.0
31077 5.0
31083 3.0
31087 15.0
31097 3.0
31101 11.0
31105 8.0
31109 619.0
31113 3.0
31115 0.0
31117 2.0
31119 0.0
31129 26.0
31141 61.0
31149 0.0
31151 33.0
31159 30.0
31161 13.0
31163 8.0
31169 1.0
31175 3.0
31179 19.0
32005 54.0
32007 60.0
32011 0.0
32033 1.0
32510 74.0
33003 101.0
33011 930.0
34009 393.0
34025 4490.0
34027 3094.0
34035 1485.0
35027 28.0
35043 194.0
35055 53

26143 67.0
27105 23.0
27123 799.0
28017 13.0
28025 8.0
28035 178.0
28047 396.0
28049 488.0
28061 39.0
28101 68.0
28105 59.0
28127 59.0
28141 16.0
28153 54.0
29055 333.0
29085 173.0
29143 319.0
29153 119.0
29181 183.0
29203 105.0
30051 1.0
31079 233.0
31091 0.0
32003 4406.0
32021 7.0
34007 1598.0
34021 1176.0
34023 3949.0
35003 0.0
35007 13.0
35009 51.0
35011 1.0
35013 477.0
35057 17.0
36085 115.0
36109 157.0
36119 4997.0
37015 28.0
37017 81.0
37063 584.0
37095 4.0
37103 34.0
37153 80.0
37185 53.0
37189 144.0
38053 15.0
38061 7.0
39095 1078.0
40055 35.0
40061 22.0
40107 29.0
40119 130.0
40135 102.0
41051 422.0
42027 460.0
42053 7.0
44007 2947.0
45031 183.0
45079 1111.0
46005 92.0
46027 30.0
46085 9.0
47027 2.0
47067 14.0
47069 25.0
47091 21.0
47135 13.0
47137 2.0
47151 40.0
47157 902.0
47175 6.0
48005 52.0
48029 3848.0
48041 902.0
48057 85.0
48087 0.0
48111 221.0
48155 1.0
48157 3076.0
48173 1.0
48191 5.0
48211 2.0
48219 29.0
48225 32.0
48245 413.0
48253 49.0
48263 13.0
48311 0.0
48315 

In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 2504
36061 4735
36047 9813 9813
36081 8899
36005 5571


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 156.0, 1003.0: 345.0, 1005.0: 69.0, 1007.0: 36.0, 1009.0: 190.0, 1011.0: 16.0, 1013.0: 76.0, 1015.0: 764.0, 1017.0: 29.0, 1019.0: 19.0, 1021.0: 81.0, 1023.0: 24.0, 1025.0: 28.0, 1027.0: 25.0, 1029.0: 34.0, 1031.0: 48.0, 1033.0: 46.0, 1035.0: 10.0, 1037.0: 23.0, 1039.0: 58.0, 1041.0: 36.0, 1043.0: 171.0, 1045.0: 61.0, 1047.0: 44.0, 1049.0: 121.0, 1051.0: 259.0, 1053.0: 31.0, 1055.0: 312.0, 1057.0: 27.0, 1059.0: 57.0, 1061.0: 22.0, 1063.0: 8.0, 1065.0: 31.0, 1067.0: 22.0, 1069.0: 107.0, 1071.0: 83.0, 1073.0: 1667.0, 1075.0: 26.0, 1077.0: 91.0, 1079.0: 39.0, 1081.0: 180.0, 1083.0: 123.0, 1085.0: 31.0, 1087.0: 69.0, 1089.0: 497.0, 1091.0: 106.0, 1093.0: 132.0, 1095.0: 252.0, 1097.0: 578.0, 1099.0: 15.0, 1101.0: 427.0, 1103.0: 210.0, 1105.0: 16.0, 1107.0: 28.0, 1109.0: 22.0, 1111.0: 40.0, 1113.0: 86.0, 1115.0: 306.0, 1117.0: 551.0, 1119.0: 32.0, 1121.0: 192.0, 1123.0: 83.0, 1125.0: 379.0, 1127.0: 166.0, 1129.0: 18.0, 1131.0: 30.0, 1133.0: 139.0, 2013.0: 1.0, 2016.0: 71.0, 2020.0: 5

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 957116
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 70.0 38921 179.85149405205416
1001 95.0 55869 170.04063076124504
1009 129.0 57826 223.08304223013872
1013 66.0 19448 339.3665158371041
1015 301.0 113605 264.9531270630694
1017 112.0 33254 336.8015877789138
1031 105.0 52342 200.60372167666503
1033 125.0 55241 226.28120417805616
1039 108.0 37049 291.50584361251316
1043 184.0 83768 219.65428325852352
1045 108.0 49172 219.63719189782802
1051 196.0 81209 241.35255944538167
1055 335.0 102268 327.5706966010873
1067 42.0 17205 244.11508282476024
1069 267.0 105882 252.16750722502405
1071 105.0 51626 203.38589083020182
1077 216.0 92729 232.93683745106708
1079 92.0 32924 279.4314178107156
1083 142.0 98915 143.55759995956123
1089 479.0 372909 128.44956812519945
1095 217.0 96774 224.23378180089694
1097 753.0 413210 182.2317949710801
1103 258.0 119679 215.576667585792
1111 41.0 22722 180.4418625121028
1113 34.0 57961 58.660133538068706
1115 230.0 89512 256.9487889891858
1117 226.0 217702 103.81163241495256
1121 167.0 79978 208.8074220410613
22

29125 7.0 8697 80.48752443371278
29127 42.0 28530 147.21345951629863
29129 2.0 3617 55.294442908487696
29131 51.0 25619 199.07100199071004
29137 11.0 8644 127.25590004627487
29139 10.0 11551 86.57259111765215
29141 42.0 20627 203.61661899452173
29145 69.0 58236 118.48341232227489
29151 14.0 13615 102.82776349614394
29157 27.0 19136 141.0953177257525
29159 77.0 42339 181.86541958950377
29161 123.0 44573 275.9518093913356
29167 39.0 32149 121.31014961585119
29171 4.0 4696 85.17887563884157
29173 12.0 10309 116.40314288485789
29177 25.0 23018 108.6106525328004
29183 462.0 402022 114.9190840302272
29186 16.0 17894 89.41544651838605
29189 2059.0 994205 207.1001453422584
29195 39.0 22761 171.34572294714644
29205 7.0 5930 118.04384485666105
29217 40.0 20563 194.52414530953655
29219 29.0 35649 81.3487054335325
29227 1.0 2013 49.67709885742673
30001 8.0 9453 84.62921823759652
30007 5.0 6237 80.1667468334135
30009 12.0 10725 111.8881118881119
30013 150.0 81366 184.35218641693092
30015 5.0 5635 8

39063 121.0 75783 159.6664159508069
39065 60.0 31365 191.2960306073649
39067 34.0 15040 226.06382978723403
39069 62.0 27006 229.57861216026066
39071 51.0 43161 118.16222979078334
39073 54.0 28264 191.05575997735636
39077 109.0 58266 187.0730786393437
39081 138.0 65325 211.25143513203216
39083 107.0 62322 171.68897018709285
39085 338.0 230149 146.86138110528395
39089 195.0 176862 110.25545340434914
39091 73.0 45672 159.83534769661938
39093 439.0 309833 141.68923258658697
39099 561.0 228683 245.31775427119638
39103 239.0 179746 132.96540674062288
39107 82.0 41172 199.16448071504905
39109 199.0 106987 186.00390701674036
39111 38.0 13654 278.30672330452614
39113 941.0 531687 176.9838269508188
39117 38.0 35328 107.56340579710145
39119 119.0 86215 138.0270254596068
39123 73.0 40525 180.1357186921653
39125 35.0 18672 187.44644387317908
39127 49.0 36134 135.60635412630762
39133 184.0 162466 113.25446554971501
39135 89.0 40882 217.69972114867178
39137 95.0 33861 280.5587549097782
39139 195.0 12

5103 58.0 23382 248.05405867761527
5105 9.0 10455 86.0832137733142
5109 22.0 10718 205.26217577906326
5139 105.0 38682 271.4440825190011
5143 342.0 239187 142.98435951786678
6009 49.0 45905 106.74218494717351
6033 43.0 64386 66.7847047494797
6035 1.0 30573 3.2708599090700945
6041 218.0 258826 84.22646874734377
6043 7.0 17203 40.69057722490263
6049 3.0 8841 33.93281303020021
6057 74.0 99755 74.18174527592602
6063 4.0 18807 21.268676556601264
6091 0.0 3005 0.0
6105 3.0 12285 24.42002442002442
6109 59.0 54478 108.30059840669628
8003 29.0 16233 178.6484322060001
8005 709.0 656590 107.98215020027719
8007 1.0 14029 7.128091809822511
8009 4.0 3581 111.7006422786931
8013 250.0 326196 76.64103790359171
8014 74.0 70465 105.01667494500816
8015 24.0 20356 117.90135586559245
8017 5.0 1831 273.07482250136536
8027 1.0 5068 19.73164956590371
8029 60.0 31162 192.54219883191067
8035 233.0 351154 66.35265439095097
8041 750.0 720403 104.10839488452991
8043 52.0 47839 108.69792428771505
8049 6.0 15734 38.1

45017 36.0 14553 247.37167594310452
45019 466.0 411406 113.27010301259583
45021 142.0 57300 247.81849912739966
45037 40.0 27260 146.7351430667645
45039 77.0 22347 344.5652660312346
45041 388.0 138293 280.5637306298945
45043 136.0 62680 216.97511167836632
45047 160.0 70811 225.95359478047197
45051 542.0 354081 153.0723196104846
45059 162.0 67493 240.02489147022655
45071 92.0 38440 239.3340270551509
46003 15.0 2751 545.2562704471102
46009 26.0 6901 376.7569917403275
46013 89.0 38839 229.1511109966786
46021 4.0 1376 290.69767441860466
46023 21.0 9292 226.00086095566078
46033 12.0 8972 133.7494427106554
46035 65.0 19775 328.6978508217446
46037 29.0 5424 534.6607669616519
46047 15.0 6713 223.44704305079696
46057 38.0 6164 616.4828033744323
46061 4.0 3453 115.84129742253113
46065 36.0 17526 205.40910647038686
46069 1.0 1301 76.86395080707149
46073 16.0 2013 794.8335817188276
46075 0.0 903 0.0
46087 24.0 5586 429.6455424274973
46091 6.0 4935 121.58054711246201
46099 337.0 193134 174.490250292

13139 414.0 204441 202.50341174226304
13191 15.0 14378 104.3260536931423
13199 67.0 21167 316.53044833939623
13209 21.0 9172 228.9576973397296
13213 77.0 40096 192.0391061452514
13217 211.0 111744 188.82445589919817
13229 53.0 19465 272.2835859234524
13247 158.0 90896 173.82503080443584
13251 25.0 13966 179.00615781182873
13253 18.0 8090 222.49690976514214
13263 17.0 6195 274.41485068603714
13277 121.0 40644 297.70691861037295
13283 23.0 6901 333.2850311549051
13319 26.0 8954 290.37301764574494
13321 65.0 20247 321.0352150935941
15001 53.0 201513 26.30103268771742
15007 1.0 72293 1.383259790021164
16031 25.0 24030 104.03662089055348
16035 12.0 8756 137.04888076747375
16037 3.0 4315 69.52491309385863
16049 17.0 16667 101.99796004079919
16057 8.0 40108 19.94614540740002
16067 30.0 21039 142.59232853272493
16075 32.0 23951 133.60611247964593
16077 9.0 7681 117.17224319750034
17069 12.0 3821 314.0539125883277
17097 1014.0 696535 145.57775273317205
17109 53.0 29682 178.5593962670979
17143 2

6069 61.0 62808 97.12138581072475
6073 3452.0 3338330 103.4049959111292
6095 177.0 447643 39.540437357447786
6113 190.0 220500 86.16780045351474
8011 21.0 5577 376.54653039268425
8031 772.0 727211 106.15901024599462
8089 63.0 18278 344.67666046613414
8105 13.0 11267 115.38120173959351
9003 2315.0 891720 259.6106401112457
11001 1041.0 705749 147.50286574972122
12011 2547.0 1952778 130.42957263959343
12063 153.0 46414 329.64191838669365
12071 900.0 770577 116.79559602739246
12095 1140.0 1393452 81.81121416453527
12097 467.0 375751 124.28443304209438
12133 48.0 25473 188.43481333176305
13001 66.0 18386 358.96878059393015
13009 117.0 44890 260.6371129427489
13025 32.0 19109 167.46035899314458
13027 44.0 15457 284.6606715404024
13033 54.0 22383 241.25452352231602
13055 68.0 24789 274.31522046068824
13059 128.0 128331 99.74207323250033
13087 66.0 26404 249.96212695046205
13089 883.0 759297 116.29178042320726
13091 100.0 20605 485.319097306479
13097 193.0 146343 131.8819485728733
13101 7.0 40

48105 15.0 3464 433.02540415704385
48115 68.0 12728 534.2551854179761
48137 4.0 1932 207.0393374741201
48165 42.0 21492 195.4215522054718
48273 84.0 30680 273.79400260756194
48291 166.0 88219 188.16808170575501
48313 27.0 14284 189.02268272192663
48329 254.0 176832 143.63916033297141
48345 8.0 1200 666.6666666666667
48409 142.0 66730 212.79784205005245
48413 6.0 2793 214.82277121374864
48439 3074.0 2102515 146.20585346596815
48443 2.0 776 257.7319587628866
48449 82.0 32750 250.38167938931298
48457 36.0 21672 166.11295681063123
48471 121.0 72971 165.81929807731836
48473 43.0 55246 77.83368931687362
48495 21.0 8010 262.17228464419475
49039 19.0 30939 61.41116390316429
51029 29.0 17148 169.11593188710054
51081 24.0 11336 211.71489061397318
51153 469.0 470335 99.71615975847004
51590 117.0 40044 292.1786035361103
51620 28.0 7967 351.44973013681437
51660 94.0 53016 177.3049645390071
51683 43.0 41085 104.66106851649019
51685 12.0 17478 68.65774116031584
51690 70.0 12554 557.5912059901227
5170

2158 3.0 8314 36.0837142169834
6025 677.0 181215 373.58938277736394
12049 34.0 26937 126.22044028659464
12051 75.0 42022 178.47794012660034
13037 14.0 6189 226.20778801098726
13265 3.0 1537 195.1854261548471
22065 36.0 10951 328.73710163455394
28055 6.0 1327 452.14770158251696
28123 71.0 28124 252.45342056606458
28135 39.0 13809 282.4245057571149
28157 27.0 8630 312.8621089223638
35031 452.0 71367 633.3459442039037
36005 5073.0 1418207 357.705186901489
37079 42.0 21069 199.34500925530403
45069 51.0 26118 195.267631518493
46121 29.0 10177 284.9562739510661
46137 9.0 2756 326.5602322206096
48047 36.0 7093 507.5426476808121
48229 11.0 4886 225.1330331559558
48323 321.0 58722 546.6435066925513
48427 281.0 64633 434.7624278619281
28021 30.0 8988 333.77837116154876
45005 15.0 8688 172.65193370165744
46102 49.0 14177 345.6302461733794
48131 39.0 11157 349.55633234740526
48505 29.0 14179 204.52782283658934
48507 39.0 11840 329.39189189189193
48127 24.0 10124 237.0604504148558
48247 15.0 5200 2

In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(1954.0, 854757, 228.60298307004214)

In [91]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 6527.160283849178,
 'ALABAMA': 17128.891622123872,
 'ALASKA': 648.4342405445556,
 'ARIZONA': 4377.754149791398,
 'ARKANSAS': 16092.460535132099,
 'CALIFORNIA': 5316.578038736096,
 'COLORADO': 7110.619646170841,
 'CONNECTICUT': 1506.9324029482761,
 'DELAWARE': 497.78734135166076,
 'FLORIDA': 11797.62215976463,
 'GEORGIA': 38949.18473236736,
 'IDAHO': 4598.2188488728,
 'ILLINOIS': 22061.018732140732,
 'INDIANA': 18643.99580727766,
 'IOWA': 23345.1479578166,
 'KANSAS': 17432.39456727567,
 'KENTUCKY': 15928.78402235842,
 'LOUISIANA': 16141.421798066547,
 'MAINE': 731.4872672284173,
 'MARYLAND': 3220.3231305844156,
 'MASSACHUSETTS': 2782.75084738473,
 'MICHIGAN': 12851.06002322183,
 'MINNESOTA': 11477.29330989526,
 'MISSISSIPPI': 23082.05606809142,
 'MISSOURI': 16185.59654486757,
 'MONTANA': 10377.037608074263,
 'NEBRASKA': 12885.269029586392,
 'NEVADA': 2003.8469927685694,
 'NEW HAMPSHIRE': 729.3843570793009,
 'NEW JERSEY': 5237.576122745338,
 'NEW YORK': 8583.614773328996,


In [92]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [93]:
Covid_death_capita[72049]

nan

In [94]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [95]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 957116.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [96]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [97]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76277412.0 1000000 10122.288889402804
ALABAMA 957116.0000000001 76277412.0 1000000 12547.830018144821
ALASKA 231145.0 76277412.0 1000000 3030.3204309029256
ARIZONA 1839932.0 76277412.0 1000000 24121.583988717393
ARKANSAS 830467.0000000003 76277412.0 1000000 10887.456433367199
CALIFORNIA 11847711.0 76277412.0 1000000 155323.9771690209
COLORADO 1337805.0 76277412.0 1000000 17538.67842291241
CONNECTICUT 874974.0 76277412.0 1000000 11470.945028916292
DELAWARE 239009.0 76277412.0 1000000 3133.417793461582
FLORIDA 3892551.999999999 76277412.0 1000000 51031.516381284666
GEORGIA 1928702.999999999 76277412.0 1000000 25285.37543984842
IDAHO 340742.0000000001 76277412.0 1000000 4467.141596256571
ILLINOIS 2987496.0000000005 76277412.0 1000000 39166.195098491284
INDIANA 1602975.9999999995 76277412.0 1000000 21015.08110946396
IOWA 699741.0 76277412.0 1000000 9173.633211362756
KANSAS 401103.0000000001 76277412.0 1000000 5258.47678209114
KENTUCKY 1416013.0000000002 76277412.0 10000

In [98]:
print (Covid_state)

{'NEW MEXICO': 3014.0, 'ALABAMA': 10000.0, 'ALASKA': 1574.0, 'ARIZONA': 14977.0, 'ARKANSAS': 4540.0, 'CALIFORNIA': 49694.0, 'COLORADO': 13791.0, 'CONNECTICUT': 7918.0, 'DELAWARE': 3296.0, 'FLORIDA': 65862.0, 'GEORGIA': 25141.0, 'IDAHO': 3692.0, 'ILLINOIS': 21759.0, 'INDIANA': 10419.0, 'IOWA': 6548.0, 'KANSAS': 3409.0, 'KENTUCKY': 11109.0, 'LOUISIANA': 7196.0, 'MAINE': 2263.0, 'MARYLAND': 10746.0, 'MASSACHUSETTS': 17246.0, 'MICHIGAN': 22495.0, 'MINNESOTA': 12182.0, 'MISSISSIPPI': 5790.0, 'MISSOURI': 50920.0, 'MONTANA': 1752.0, 'NEBRASKA': 3666.0, 'NEVADA': 5266.0, 'NEW HAMPSHIRE': 3048.0, 'NEW JERSEY': 45365.0, 'NEW YORK': 74023.0, 'NORTH CAROLINA': 21629.0, 'NORTH DAKOTA': 1144.0, 'OHIO': 20329.0, 'OKLAHOMA': 7627.0, 'OREGON': 3745.0, 'PENNSYLVANIA': 32663.0, 'RHODE ISLAND': 4403.0, 'SOUTH CAROLINA': 15447.0, 'SOUTH DAKOTA': 2168.0, 'TENNESSEE': 13522.0, 'TEXAS': 67724.0, 'UTAH': 6609.0, 'VERMONT': 1611.0, 'VIRGINIA': 18007.0, 'WASHINGTON': 8513.0, 'WEST VIRGINIA': 3703.0, 'WISCONSIN':

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [99]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [100]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [101]:
Total_covid_death_cap


{'NEW MEXICO': 6527.160283849178,
 'ALABAMA': 17128.891622123872,
 'ALASKA': 648.4342405445556,
 'ARIZONA': 4377.754149791398,
 'ARKANSAS': 16092.460535132099,
 'CALIFORNIA': 5316.578038736096,
 'COLORADO': 7110.619646170841,
 'CONNECTICUT': 1506.9324029482761,
 'DELAWARE': 497.78734135166076,
 'FLORIDA': 11797.62215976463,
 'GEORGIA': 38949.18473236736,
 'IDAHO': 4598.2188488728,
 'ILLINOIS': 22061.018732140732,
 'INDIANA': 18643.99580727766,
 'IOWA': 23345.1479578166,
 'KANSAS': 17432.39456727567,
 'KENTUCKY': 15928.78402235842,
 'LOUISIANA': 16141.421798066547,
 'MAINE': 731.4872672284173,
 'MARYLAND': 3220.3231305844156,
 'MASSACHUSETTS': 2782.75084738473,
 'MICHIGAN': 12851.06002322183,
 'MINNESOTA': 11477.29330989526,
 'MISSISSIPPI': 23082.05606809142,
 'MISSOURI': 16185.59654486757,
 'MONTANA': 10377.037608074263,
 'NEBRASKA': 12885.269029586392,
 'NEVADA': 2003.8469927685694,
 'NEW HAMPSHIRE': 729.3843570793009,
 'NEW JERSEY': 5237.576122745338,
 'NEW YORK': 8583.614773328996,


# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [102]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [103]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [104]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [105]:
#print (Medicaid_demand)

In [106]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [107]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [108]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [109]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)